In [49]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is

In [51]:
from ml.models.base.tabular_dataset import TabularDataset
from ml.models.base.brisk_model_lightning import BriskDataModule, TabularDataset
from ml.preprocess import data
from utils import dasker, helper

In [30]:

from torch.utils.data import random_split, DataLoader, Dataset
from dask_ml.preprocessing import StandardScaler

import torch
from torchmetrics import Accuracy, MeanSquaredError
from torch import nn
import torch.optim as optim

# import pytorch_lightning as pl
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np


In [80]:
from sklearn.model_selection import train_test_split

X_train_orig, y = data.get_dataset(f_name = '20220319_covid_merge_processed.csv' , sep = ',')
# X_tilda = X.drop(brisk_xgboost.DROP_LIST, axis = 1)
X = X_train_orig.drop(['location'], axis = 1)

X_scalar = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scalar, y, test_size=0.25, random_state=123)

X_test_tensor = helper.df_to_tensor(X_test)
# y_test_tensor = helper.df_to_tensor(y_test)
# y_test_tensor = y_test_tensor.reshape(y_test_tensor.shape[0], )


In [6]:


def get_dataset():
    df = pd.read_csv('/mnt/c/Users/rwmas/GitHub/xai/xai_api/app/test/data/20220319_covid_merge_processed.csv', sep=",")

    X = df[df.columns[df.columns!='y']]

    X = X.drop(['location'], axis = 1)

    y = df[df.columns[df.columns=='y']]

    return X, y


In [86]:
N_LAYERS = 5
MAX_N_NEURONS = 256
EPOCHS = 100
N_TRIALS = 100

TEMP_PATH = "/mnt/c/Users/rwmas/GitHub/xai/xai_api/app/ml/models/saved/temp/"
MODEL_SAVE_PATH = "/mnt/c/Users/rwmas/GitHub/xai/xai_api/app/ml/models/saved/base/brisk_nn_pytorch/"

DROP_LIST = ['location']


In [97]:

def define_model(trial, input_dim):

    # We optimize the number of layers, hidden untis and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, N_LAYERS)
    layers = []

    in_features = input_dim

    # layers.append(nn.BatchNorm1d(num_features = in_features))
    # layers.append(nn.ReLU())
    
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 1, MAX_N_NEURONS)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        # p = trial.suggest_uniform("dropout_l{}".format(i), 0.1, 0.5)
        # layers.append(nn.Dropout(p))

        in_features = out_features

    layers.append(nn.Linear(in_features, 1))
    layers.append(nn.ReLU())
    # layers.append(torch.nn.Flatten(0, 1))
    # layers.append(nn.LogSoftmax(dim=1))

    model = nn.Sequential(*layers)

    return model

In [99]:
# Train and evaluate the accuracy of neural network with the addition of pruning mechanism
def train_and_evaluate(param, model, trial, df_X, df_y):
    
    X_base, X_val, y_base, y_val = train_test_split(df_X, df_y, random_state=0, shuffle=True)

    # train_data, val_data = train_test_split(df_, test_size = 0.2, random_state = 42)
    train, val = TabularDataset(X_base, y_base), TabularDataset(X_val, y_val)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=param['batch_size'], shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=param['batch_size'])

    # train_dataloader = BriskDataModule(X_base, y_base, batch_size=8)
    # val_dataloader = BriskDataModule(X_val, y_val, batch_size=8)

    # train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    # val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    # use_cuda = torch.cuda.is_available()
    # device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.MSELoss()
    optimizer = getattr(optim, param['optimizer'])(model.parameters(), lr= param['learning_rate'])

#    if use_cuda:

#            model = model.cuda()
#            criterion = criterion.cuda()

    for epoch_num in range(EPOCHS):

            total_err_train = 0
            total_loss_train = 0

            for train_x, train_y in train_dataloader:

                # train_label = train_label.to(device)
                # train_input = train_input.to(device)
                # train_y = train_y.reshape(train_y.shape[0], )
                output = model(train_x.float())
                
                print(f'train: {output.shape}')
                # print(f'y: {train_y.shape}')

                batch_loss = criterion(output, train_y)
                # print(batch_loss)
                total_loss_train += batch_loss.item()
                
                # acc = (output.argmax(dim=1) == train_label).sum().item()

                #err = np.round_(mean_squared_error(output, train_y), decimals=2, out=None)
                # total_err_train += err

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_err_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_x, val_y in val_dataloader:

                    # val_label = val_label.to(device)
                    # val_input = val_input.to(device)
                    # val_y = val_y.reshape(val_y.shape[0], )
                    output = model(val_x.float())
                    print(f'val: {output.shape}')

                    batch_loss = criterion(output, val_y)
                    total_loss_val += batch_loss.item()
                    
                    # acc = (output.argmax(dim=1) == val_label).sum().item()
                    # err = np.round_(mean_squared_error(output, val_y), decimals=2, out=None)
                    #total_acc_val += err
            
            accuracy = total_loss_val
            
            # Add prune mechanism
            trial.report(accuracy, epoch_num)

#            if trial.should_prune():
#                raise optuna.exceptions.TrialPruned()

    return accuracy

In [100]:
import pickle

def objective(trial):

     params = {
          'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
          'optimizer': trial.suggest_categorical("optimizer", ["Adam", "Adamax", 'NAdam']),
          'batch_size': trial.suggest_categorical("batch_size", [8, 16, 32, 64, 128, 512, 1024, 2048])
          }

     df_X, df_y = get_dataset()

     df_X = StandardScaler().fit_transform(df_X)     

     input_shape = df_X.shape[1]     

     model = define_model(trial, input_shape)

     accuracy = train_and_evaluate(params, model, trial, df_X, df_y)

     with open(TEMP_PATH+"{}.pickle".format(trial.number), "wb") as fout:
        pickle.dump(model, fout)

     return accuracy
  

In [101]:
import optuna
import dask_optuna
import joblib

# EPOCHS = 30

# with Client() as client:
client = dasker.get_dask_client()
print(f"Dask dashboard is available at {client.dashboard_link}")

storage = dask_optuna.DaskStorage()
study = optuna.create_study(storage=storage, direction="minimize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner())

with joblib.parallel_backend("dask"):
    study.optimize(objective, n_trials=3, n_jobs=-1)

print("Number of trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial


[I 2022-09-28 17:39:08,767] A new study created in memory with name: no-name-2f98c17e-c50d-408d-ad61-4feb17e9f9a7


Dask dashboard is available at http://127.0.0.1:8787/status
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])train: torch.Size([8, 1])

train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([143, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
val: torch.Size([48, 1])
train: torch.Size([8, 1])
train: torch.Size([7, 1])
val: torch.Size([8, 1])
train: torch.Size([8, 1])
val: torch.Size([8, 1])
val: torch.Size([8, 1])
train: torch.Size([8, 1])
val: torc

[I 2022-09-28 17:39:16,535] Trial 1 finished with value: 4160406.25 and parameters: {'learning_rate': 6.366733078974486e-05, 'optimizer': 'Adamax', 'batch_size': 1024, 'n_layers': 4, 'n_units_l0': 165, 'n_units_l1': 227, 'n_units_l2': 26, 'n_units_l3': 140}. Best is trial 1 with value: 4160406.25.


train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([7, 1])
train: torch.Size([8, 1])
val: torch.Size([8, 1])
train: torch.Size([8, 1])
val: torch.Size([8, 1])
val: torch.Size([8, 1])
train: torch.Size([8, 1])
val: torch.Size([8, 1])
val: torch.Size([8, 1])
train: torch.Size([8, 1])
val: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([7, 1])
train: torch.Size([8, 1])
val: torch.Size([8, 1])
val: torch.Size([8, 1])
train: torch.Size([8, 1])
val: torch.Size([8, 1])
train: torch.Size([8, 1])
val: torch.Size([8, 1])
val: t

[I 2022-09-28 17:39:20,600] Trial 2 finished with value: 24867253.25 and parameters: {'learning_rate': 1.556308562229793e-05, 'optimizer': 'NAdam', 'batch_size': 8, 'n_layers': 3, 'n_units_l0': 17, 'n_units_l1': 111, 'n_units_l2': 97}. Best is trial 1 with value: 4160406.25.


train: torch.Size([8, 1])
train: torch.Size([7, 1])
val: torch.Size([8, 1])
val: torch.Size([8, 1])
val: torch.Size([8, 1])
val: torch.Size([8, 1])
val: torch.Size([8, 1])
val: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([7, 1])
val: torch.Size([8, 1])
val: torch.Size([8, 1])
val: torch.Size([8, 1])
val: torch.Size([8, 1])
val: torch.Size([8, 1])
val: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: torch.Size([8, 1])
train: tor

[I 2022-09-28 17:39:22,467] Trial 0 finished with value: 5234823.4375 and parameters: {'learning_rate': 0.0005693338866363375, 'optimizer': 'Adam', 'batch_size': 8, 'n_layers': 4, 'n_units_l0': 52, 'n_units_l1': 62, 'n_units_l2': 249, 'n_units_l3': 211}. Best is trial 1 with value: 4160406.25.


In [102]:
best_trial = study.best_trial

for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))


learning_rate: 6.366733078974486e-05
optimizer: Adamax
batch_size: 1024
n_layers: 4
n_units_l0: 165
n_units_l1: 227
n_units_l2: 26
n_units_l3: 140


In [103]:
# Load the best model.
with open(TEMP_PATH+"{}.pickle".format(study.best_trial.number), "rb") as fin:
    best_clf = pickle.load(fin)

In [104]:
best_clf.eval()

Sequential(
  (0): Linear(in_features=40, out_features=165, bias=True)
  (1): ReLU()
  (2): Linear(in_features=165, out_features=227, bias=True)
  (3): ReLU()
  (4): Linear(in_features=227, out_features=26, bias=True)
  (5): ReLU()
  (6): Linear(in_features=26, out_features=140, bias=True)
  (7): ReLU()
  (8): Linear(in_features=140, out_features=1, bias=True)
  (9): ReLU()
)

In [105]:
pred = best_clf(X_test_tensor)

In [106]:
mean_squared_error(helper.torch_tensor_to_numpy(pred), y_test)

2354808.990135021

In [107]:


with open("temp.pickle".format(trial.number), "wb") as fout:
    pickle.dump(best_clf, fout)


In [83]:
df = pd.DataFrame(list(zip(X_train_orig['location'].values, y_test.values.reshape(y_test.shape[0], ), helper.torch_tensor_to_numpy(pred) )),
               columns =['countary', 'deaths', 'predictions'])
df



,countary,deaths,predictions
0,0,0.000,126.535271
1,1,4896.644,4304.256348
2,2,1486.844,1670.905273
3,3,55.799,101.539177
4,4,153.473,294.491791
5,5,2252.476,2243.651367
6,6,146.353,64.246170
7,7,0.000,155.027802
8,8,604.743,459.783722
9,9,45.966,74.649467
